In [1]:
# Load necessar libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import numpy as np
import re


In [2]:
#Loading the heart_disease_uci dataset
train_data = pd.read_csv('..\data\heart_disease\data_train.csv')
test_data = pd.read_csv('..\data\heart_disease\data_test.csv')
train_data.head()



,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,881,62,1.0,3.0,0.0,146.12,170.00,0.0,2.0,120.00,1.0,3.000,0.0,0.0,2.0,1
1,458,54,1.0,1.0,2.0,150.00,216.98,0.0,1.0,122.00,0.0,0.000,2.0,0.0,2.0,0
2,798,51,1.0,3.0,2.0,134.86,339.00,0.0,1.0,132.41,1.0,2.943,1.0,0.0,2.0,1
3,26,50,0.0,0.0,2.0,120.00,219.00,0.0,1.0,158.00,0.0,1.600,1.0,0.0,1.0,0
4,85,52,1.0,0.0,1.0,120.00,325.00,0.0,1.0,172.00,0.0,0.200,2.0,0.0,1.0,0


In [3]:
#Splitting the data into features and target
X_train = train_data.iloc[:,:-1]
y_train = train_data.iloc[:,-1]

X_test = test_data.iloc[:,:-1]
y_test = test_data.iloc[:,-1]
X_train.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal
0,881,62,1.0,3.0,0.0,146.12,170.00,0.0,2.0,120.00,1.0,3.000,0.0,0.0,2.0
1,458,54,1.0,1.0,2.0,150.00,216.98,0.0,1.0,122.00,0.0,0.000,2.0,0.0,2.0
2,798,51,1.0,3.0,2.0,134.86,339.00,0.0,1.0,132.41,1.0,2.943,1.0,0.0,2.0
3,26,50,0.0,0.0,2.0,120.00,219.00,0.0,1.0,158.00,0.0,1.600,1.0,0.0,1.0
4,85,52,1.0,0.0,1.0,120.00,325.00,0.0,1.0,172.00,0.0,0.200,2.0,0.0,1.0


In [4]:
#Scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Converting the data into dataframes
X_train = pd.DataFrame(X_train, columns = train_data.columns[:-1])
X_test = pd.DataFrame(X_test, columns = test_data.columns[:-1])


In [5]:
import os
from dotenv import load_dotenv
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM


load_dotenv()  # Load environment variables from .env file
token = os.getenv("HF_TOKEN")

# Load the Mistral model and tokenizer from Hugging Face
model_name = "mistralai/Mistral-7B-v0.1"

# Ensure compatibility by using AutoTokenizer and AutoModel
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token, device_map="auto")
model = AutoModelForCausalLM.from_pretrained(model_name, num_labels=2,token=token,device_map="auto")

# Ensure pad_token_id is set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [13]:
# Function to create the prompt for in-context learning
# Create a new function for constructing prompts
def create_prompt(X_train, y_train, test_row):
    """
    This function generates a prompt for the model by providing examples
    of input features and their corresponding labels from the training set, followed by
    the test instance for which we want the model to predict the label.
    """
    prompt = ""

    # Add the test instance for prediction
    test_features = ", ".join([f"{col}={test_row[col]}" for col in X_train.columns])
    prompt += f"Input: {test_features} -> Output: ? \n"  

    # Add examples from the training set to the prompt
    for i, train_row in X_train.iterrows():
        input_features = ", ".join([f"{col}={train_row[col]}" for col in X_train.columns])
        label = y_train.iloc[i]
        prompt += f"Input: {input_features} -> Output: {label}\n"
    
    return prompt

# Example: Create a prompt for the first test instance
prompt = create_prompt(X_train, y_train, X_test.iloc[0])  # Correct usage of .iloc
print(prompt)



Input: id=-0.532059655920466, age=-1.9174056266954647, sex=0.5059374430161635, dataset=-0.20703006515429206, cp=0.22730208119306572, trestbps=-0.6442471951163501, chol=-0.3036881424415377, fbs=-0.4011025813393599, restecg=0.029907157263792106, thalch=1.7000194852984174, exang=-0.815572419249648, oldpeak=-0.8513851852119763, slope=1.2063802017462582, ca=-0.6443687350014317, thal=-0.7265159406448333 -> Output: ? 
Input: id=1.5822507045174068, age=0.9024125467171098, sex=0.5059374430161635, dataset=1.564762120352207, cp=-0.8182874922950367, trestbps=0.7455525400597492, chol=-0.26708872490983715, fbs=-0.4011025813393599, restecg=1.6021691391317174, thalch=-0.6550501357991567, exang=1.22613268472226, oldpeak=1.7593009477347854, slope=-2.0048162846379953, ca=-0.6443687350014317, thal=0.8369803526727027 -> Output: 1
Input: id=-0.011961920197780689, age=0.03477618566708685, sex=0.5059374430161635, dataset=-0.20703006515429206, cp=1.2728916546811682, trestbps=0.9520005864794758, chol=0.16277143

In [16]:

def extract_prediction(generated_text, prompt):
    """
    Extracts the predicted label from the generated text.
    Assumes the generated output follows the pattern "Input: <features> -> Output: <prediction>".
    """
    try:
        # Use regular expression to find the value after "output="
        #match = re.search(r'(Output|class|target)\s*[:=]\s*([-+]?\d*\.\d+|\d+)', generated_text)
        match = re.search(r'Output\s*:\s*([-+]?\d*\.\d+|\d+)', generated_text)
        if match:
            prediction = match.group(1)
            print(prediction)
            return (int(round(float(prediction))) if prediction.replace('.', '', 1).isdigit() else None)
        else:
            return None
    except Exception as e:
        print(f"Error extracting prediction: {e}")
        return None

In [17]:
from transformers import set_seed
# Function to generate predictions for a batch of test data
def in_context_learning(X_train, y_train, X_test, model, tokenizer):
    """
    Generates predictions for the test set using in-context learning.
    """
    # Set the seed for reproducibility
    # set_seed(42)
    
    
    predictions = []
    temp = []
    for _, test_row in X_test.iterrows():
        prompt = create_prompt(X_train, y_train, test_row)
        
        # Tokenize the prompt and generate output
        inputs = tokenizer(prompt, return_tensors="pt",truncation=True, max_length=300)
        input_length = inputs['input_ids'].shape[1]
        output = model.generate(**inputs,min_length=input_length, max_new_tokens=50, do_sample=True, temperature=0.7, top_k=50,) # Adjust parameters as needed, these are working parameters
        
        # Decode the generated output
        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        temp.append(generated_text)
        # Extract the prediction from the generated text
        predicted_label = extract_prediction(generated_text, prompt)
        predictions.append(predicted_label)
    
    return predictions, temp




# Generate predictions for the test set
batch_size = 8  # Adjust batch size as needed
predictions = []
temp = []
pd.DataFrame(predictions).to_csv('results/mistral_main.csv', index=False)
pd.DataFrame(temp).to_csv('results/textmistral.csv', index=False)

for i in range(0, len(X_test), batch_size):
    X_batch = X_test[i:i + batch_size]
    batch_predictions, textsp = in_context_learning(X_train, y_train, X_batch, model, tokenizer)
    predictions.extend(batch_predictions)
    temp.extend(textsp)
    
# Append the predictions to the results folder in mistral.csv file
pd.DataFrame(predictions).to_csv('results/mistral_main.csv', mode='a', header=False, index=False)

# Append the text to the results folder in textmistral.csv file
pd.DataFrame(temp).to_csv('results/textmistral.csv', mode='a', header=False, index=False)






Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1.0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0.0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


0.0921937714


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


## This part of the code might throw an error if the predictions donot match the specified format in this case manual work to scrape the predictions must be performed from the generated csv 

In [ ]:
# Evaluate the Mistral model with confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report

# Calculate confusion matrix
confusion = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")

# Display confusion matrix
print(confusion)

# Calculate classification report
report = classification_report(y_test, predictions)
print("\nClassification Report:")
# Display classification report
print(report)

# Calculate accuracy, precision, recall, and F1 score
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

# Display metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
